# Doc2Vecの実行条件調査（part2）

- ラベル毎にユニークな教師データを設定（このためテストデータから学習セットの大半が失われます）


- 一部品詞は落とす


- パラメータを一部変更（DBOWを使用／feature数を調整）


accuracyは 0.919 と向上した様子ですが、品詞を落とさないようにしたケースより低下しています。

## (1) テストデータ／環境準備

In [1]:
'''
    テスト環境を準備するためのモジュールを使用します。
'''
import sys
import os
learning_dir = os.path.abspath("../../") #<--- donusagi-bot/learning
os.chdir(learning_dir)

if learning_dir not in sys.path:
    sys.path.append(learning_dir)

## (2) Doc2Vecの動作確認

### (2-1) コーパス生成

コーパス（単語が半角スペースで区切られた文字列）生成時、一部の品詞を落とすようにします。

（＝learning.core.nlang.Nlang クラスの仕様に従います）

In [2]:
import numpy as np

from learning.core.learn.learning_parameter import LearningParameter
from learning.core.datasource import Datasource

_bot_id = 9  # bot_id = 9はセプテーニ
attr = {
    'include_failed_data': False,
    'include_tag_vector': False,
    'classify_threshold': 0.5,
    'algorithm': LearningParameter.ALGORITHM_LOGISTIC_REGRESSION,
    'params_for_algorithm': {'C': 140},
    'excluded_labels_for_fitting': None
}

learning_parameter = LearningParameter(attr)

In [3]:
_datasource = Datasource(type='csv')
learning_training_messages = _datasource.learning_training_messages(_bot_id)
questions = np.array(learning_training_messages['question'])
answer_ids = np.array(learning_training_messages['answer_id'])

2017/05/18 PM 12:02:52 ['./fixtures/learning_training_messages/benefitone.csv', './fixtures/learning_training_messages/ptna.csv', './fixtures/learning_training_messages/septeni.csv', './fixtures/learning_training_messages/toyotsu_human.csv']
2017/05/18 PM 12:02:52 ['./fixtures/question_answers/toyotsu_human.csv']


In [4]:
from learning.core.nlang import Nlang

_sentences = np.array(questions)
_separated_sentences = Nlang.batch_split(_sentences)

#### 試しに、学習セットをラベル毎にユニーク化してみます。

In [5]:
unique_answer_ids = []
unique_questions = []
unique_separated_sentences = []

for index, answer_id in enumerate(answer_ids):
    if answer_id not in unique_answer_ids:
        '''
            データをリストに格納
        '''
        unique_answer_ids.append(answer_id)
        unique_questions.append(questions[index])
        unique_separated_sentences.append(_separated_sentences[index])

In [6]:
'''
    教師データを表示
'''
for index, answer_id in enumerate(unique_answer_ids):
    print(answer_id, unique_separated_sentences[index])

4420 Ｗｉｎｄｏｗｓ ログイン 出来る ない
2761 ブルース クリーン 発生 する 強制 的 シャット ダウン する れる
4423 パソコン メモリ 増設 する
4425 共有 ＰＣ 設定 する 管理 者 権限 他 ユーザー 付与 する せる
4428 データ メディア コピー する
4429 Ｍａｃ 使用 する
4432 誤る 送信 防止 システム インストール 出来る ない
4434 ａｋｉｎｄｏ 接続 できる ない
4437 サイボウズ ログイン できる ない
4441 販管 （ ＳＡＰ ） パスワード 忘れる
4444 拠点 ＩＰ アドレス 知る
4445 プリンタードライバ インストール 出来る ない
4447 ＰＣ 複数 台 使用 する
4450 Ｃ ドライブ 容量 枯渇
4454 Ｃ ドライブ 容量 不足
4455 複数 メールアドレス アカウント 欲しい
4530 マウス 壊れる
4458 マウス 破損
4459 マウス 交換
4610 共有 フォルダ アクセス できる ない
4460 共有 フォルダ アクセス
4468 リモート ＶＰＮ 接続 出来る ない
4472 リモート アクセス 出来る ない
4473 ＶＰＮ 接続 出来る ない
4474 海外 出張 ＰＣ 持つ
4478 ＭＬ 届く ない
7084 ＰＣ 破損 する
4480 ＰＣ 破損
4484 ＰＣ 不具合 起こす
4489 Ｏｆｆｉｃｅ ファイル ＰＤＦ 化 する
4491 ＩＤ システム 導入 前 事前 設定 ５ ＿ 販管 （ ＳＡＰ ） 利用 手順
4492 ＩＤ システム 導入 後 設定 Ｇ メール ログイン 手順 ． ｐｄｆ
4500 Ｇｍａｉｌ 手順
4493 無線 設定 する （ ａｋｉｎｄｏ ｈｉｎｅｒａｎｋａｉ − ７ ） 繋がる ない
4494 ａｋｉｎｄｏ 繋がる ない
4495 ｈｉｎｅｒａｎｋａｉ − ７ 繋がる ない
4496 サイボウズ パスワード 変更 する 社外 場合
4497 サイボウズ パスワード 変更
4498 ＳＦＴＳ ２ 使い方 知る
4499 ＩＤ システム 導入 前 事前 設定 ６ ＿ ＣＲＭ 利用 手順
4501 社外 サイボウズ 利用
4502 販管 文字 入力 する 一文字 入力 できる ない
4503 販管 ログイン パ

### (2-2) コーパスにタグ付け

models.doc2vecの仕様に従います。

In [7]:
from gensim import models
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

def doc_to_sentence(sentences, name):
    words = sentences.split(' ')
    return TaggedDocument(words=words, tags=[name])

def corpus_to_sentences(separated_sentences, answer_ids):
    for idx, (doc, name) in enumerate(zip(separated_sentences, answer_ids)):
        yield doc_to_sentence(doc, name)

### (2-3) 学習処理／モデルのシリアライズ

In [8]:
'''
    ユニークになったコーパス／ラベルから、
    学習セットを生成する
    
    教師データ件数＝173
'''
sentences = corpus_to_sentences(unique_separated_sentences, unique_answer_ids)
sentence_list = list(sentences)
len(sentence_list)

173

In [9]:
'''
    パラメータ変更点：
    　学習時にDBOWを使用する
    　featureの数をサンプル件数に近い値に設定
'''
model = Doc2Vec(dm=0, size=200, min_count=1, iter=1000)

In [10]:
'''
    ボキャブラリ生成／学習実行
    学習モデルは、ファイルに保存しておく
'''
model.build_vocab(sentence_list)
model.train(sentence_list)

model_path = 'prototype/better_algorithm/doc2vec.model'
model.save(model_path)

In [11]:
'''
    モデル内に保持されているベクトルの数を取得
    （教師データ数と同じであることを確認）
'''
len(model.docvecs)

173

In [12]:
model.docvecs

### (2-4) 予測処理

質問文は一部「揺れ」を与えたものとなっております

In [13]:
model_path = 'prototype/better_algorithm/doc2vec.model'

def predict(word, model_path):
    '''
        予測処理にかけるコーパスを生成
        （学習セット作成時と同じ関数を使用）
    '''
    corpus = Nlang.split(word).split()

    '''
        コーパスからベクトルを生成し、
        ロードしたモデルから類似ベクトルを検索
    '''
    loaded_model = models.Doc2Vec.load(model_path)
    inferred_vector = loaded_model.infer_vector(corpus)
    ret = loaded_model.docvecs.most_similar([inferred_vector])

    return corpus, ret

In [14]:
'''
    マウスが破損（正解＝4458）
'''
predict('マウスが破損', model_path)

(['マウス', '破損'],
 [(4458, 0.8315008878707886),
  (4480, 0.8142363429069519),
  (7084, 0.8142303824424744),
  (4530, 0.7549080848693848),
  (4459, 0.7379305362701416),
  (4558, 0.6827746629714966),
  (4564, 0.6692854762077332),
  (4474, 0.6053973436355591),
  (4598, 0.5804398655891418),
  (4562, 0.5746040344238281)])

In [15]:
'''
    無線を使用したい（正解＝4516）
'''
predict('無線を使用したい', model_path)

(['無線', '使用', 'する'],
 [(4516, 0.8753690719604492),
  (4521, 0.8112785816192627),
  (4592, 0.7466295957565308),
  (4577, 0.733330249786377),
  (4429, 0.6924296617507935),
  (4447, 0.688360869884491),
  (4494, 0.6483550667762756),
  (4493, 0.6413534879684448),
  (4623, 0.6363179087638855),
  (4594, 0.6331053972244263)])

In [16]:
'''
    情報システムのアドレス（正解＝7040）
'''
predict('情報システムのアドレス', model_path)

(['情報', 'システム', 'アドレス'],
 [(7065, 0.7657581567764282),
  (7040, 0.752773106098175),
  (4444, 0.7101176381111145),
  (4500, 0.6918478012084961),
  (4539, 0.681902289390564),
  (7064, 0.6811095476150513),
  (4513, 0.628372073173523),
  (4558, 0.6119155287742615),
  (4562, 0.6076346635818481),
  (4577, 0.6010578870773315)])

In [17]:
'''
    Office2010を使用したいのですが（正解＝4625）
'''
predict('Office2010を使用したいのですが', model_path)

(['Ｏｆｆｉｃｅ', '２０１０', '使用', 'する'],
 [(4625, 0.8615430593490601),
  (4523, 0.8189694285392761),
  (4590, 0.718734622001648),
  (7068, 0.7127879858016968),
  (4533, 0.708860456943512),
  (4489, 0.6973989009857178),
  (4595, 0.6361585855484009),
  (4598, 0.6288830041885376),
  (4536, 0.6134272813796997),
  (4562, 0.6077337861061096)])

In [18]:
'''
    携帯からサイボウズを使いたいのですが、どうしたら出来ますか？
    （正解＝4504だが、学習時の教師データから抜け落ちたもの）
'''
predict('携帯からサイボウズを使いたいのですが、どうしたら出来ますか？', model_path)

(['携帯', 'サイボウズ', '使う', 'どう', 'する', '出来る'],
 [(4533, 0.7207569479942322),
  (4420, 0.6997624635696411),
  (4515, 0.6950582265853882),
  (4437, 0.6950013041496277),
  (7056, 0.6923589110374451),
  (4473, 0.6753672361373901),
  (4468, 0.6658152341842651),
  (4501, 0.6581986546516418),
  (4445, 0.6449718475341797),
  (4625, 0.6444051265716553)])

## (3) accuracy 測定

ここからが本題です

In [19]:
model_path = 'prototype/better_algorithm/doc2vec.model'

len(unique_separated_sentences)

173

In [20]:
def predict_similarity(separated_sentence, model_path):
    corpus = separated_sentence.split()
    loaded_model = models.Doc2Vec.load(model_path)
    inferred_vector = loaded_model.infer_vector(corpus)
    ret = loaded_model.docvecs.most_similar([inferred_vector])

    answer_id, similarity = ret[0]
    return corpus, answer_id, similarity

In [21]:
def get_prediction_statistics(separated_sentences, answer_ids, model_path):
    '''
        学習セットの質問文をそのまま予測処理にかけて、
        回答を予測
    '''
    statistics = []
    for i, _ in enumerate(separated_sentences):
        sentence = separated_sentences[i]
        preferred_answer_id = answer_ids[i]
        corpus, answer_id, similarity = predict_similarity(separated_sentences[i], model_path)
        corpus_len = len(corpus)
        statistics.append((i, corpus_len, preferred_answer_id, answer_id, similarity))

    return statistics

In [22]:
prediction_statistics = get_prediction_statistics(unique_separated_sentences, unique_answer_ids, model_path)

In [23]:
ncorrect_by_corpus_len = {}
nsample_by_corpus_len = {}

ncorrect_by_answer_id = {}
nsample_by_answer_id = {}
corpus_len_by_answer_id = {}

ncorrect = 0
nsample = 0

'''
    予測結果を、質問文の単語数毎／回答ID毎に統計する
'''
for statistics in prediction_statistics:
    i, corpus_len, preferred_answer_id, answer_id, similarity = statistics
    
    '''
        質問文の単語数ごとに統計を取る
    '''
    if corpus_len not in nsample_by_corpus_len.keys():
        ncorrect_by_corpus_len[corpus_len] = 0
        nsample_by_corpus_len[corpus_len] = 0
    nsample_by_corpus_len[corpus_len] += 1

    '''
        回答IDごとに統計を取る
    '''
    if preferred_answer_id not in nsample_by_answer_id.keys():
        ncorrect_by_answer_id[preferred_answer_id] = 0
        nsample_by_answer_id[preferred_answer_id] = 0
        corpus_len_by_answer_id[preferred_answer_id] = 0
    nsample_by_answer_id[preferred_answer_id] += 1
    corpus_len_by_answer_id[preferred_answer_id] += corpus_len
    
    '''
        正解かどうか検査
    '''
    nsample += 1
    if preferred_answer_id == answer_id:
        ncorrect += 1
        ncorrect_by_corpus_len[corpus_len] += 1
        ncorrect_by_answer_id[preferred_answer_id] += 1

In [24]:
'''
    質問文の単語数ごとの統計情報を編集
'''
info_by_corpus_len = []
for k, v in ncorrect_by_corpus_len.items():
    info_by_corpus_len.append((
        k, 
        ncorrect_by_corpus_len[k]/nsample_by_corpus_len[k], 
        ncorrect_by_corpus_len[k], 
        nsample_by_corpus_len[k]
    ))

In [25]:
'''
    回答IDごとの統計情報を編集
'''
info_by_answer_id = []
for k, v in ncorrect_by_answer_id.items():
    info_by_answer_id.append((
        k, 
        ncorrect_by_answer_id[k]/nsample_by_answer_id[k], 
        ncorrect_by_answer_id[k], 
        nsample_by_answer_id[k],
        corpus_len_by_answer_id[k]/nsample_by_answer_id[k]
    ))

In [26]:
'''
    全体の正解率
'''
print("accuracy=%0.3f (%d/%d)" % (
    ncorrect/nsample, ncorrect, nsample))

accuracy=0.919 (159/173)


In [27]:
'''
    質問文の単語数ごとの正解率をリスト
'''
for info in info_by_corpus_len:
    print("word_count=%2d: accuracy=%0.3f (%d/%d)" % (
        info[0], info[1], info[2], info[3]
    ))

word_count= 1: accuracy=1.000 (1/1)
word_count= 2: accuracy=0.846 (11/13)
word_count= 3: accuracy=0.917 (22/24)
word_count= 4: accuracy=0.871 (27/31)
word_count= 5: accuracy=0.935 (29/31)
word_count= 6: accuracy=0.941 (16/17)
word_count= 7: accuracy=0.909 (10/11)
word_count= 8: accuracy=1.000 (14/14)
word_count= 9: accuracy=1.000 (9/9)
word_count=10: accuracy=1.000 (7/7)
word_count=11: accuracy=0.714 (5/7)
word_count=12: accuracy=1.000 (3/3)
word_count=13: accuracy=1.000 (1/1)
word_count=14: accuracy=1.000 (2/2)
word_count=15: accuracy=1.000 (1/1)
word_count=16: accuracy=1.000 (1/1)


In [28]:
'''
    正解でなかったラベルの一覧。

    参考情報として、回答ラベルがつけられている
    質問文の単語数（word count）を表示
'''
sorted_info_by_answer_id = sorted(info_by_answer_id, key=lambda x:x[1], reverse=False)
for info in sorted_info_by_answer_id:
    if info[1] > 0:
        continue #当たりは表示しない
    print("answer_id=%4d: word count=%2d" % (
        info[0], info[4]
    ))

answer_id=4629: word count= 4
answer_id=4480: word count= 2
answer_id=7084: word count= 3
answer_id=4454: word count= 4
answer_id=4459: word count= 2
answer_id=4468: word count= 5
answer_id=4492: word count=11
answer_id=4493: word count=11
answer_id=4501: word count= 3
answer_id=4503: word count= 4
answer_id=4510: word count= 5
answer_id=4518: word count= 7
answer_id=4546: word count= 4
answer_id=4605: word count= 6
